In [19]:
import numpy as np
import scipy.io as sio
from PIL import Image
from numpy import linalg as LA

In [3]:
mat_contents = sio.loadmat("mnist_digits.mat") # mat_contents['X'].shape = (10000, 784)

In [79]:
# select any sample from the 10,000 dataset
n = 279
# to see the Y value
print(mat_contents['Y'][n][0])
# to see the X value as image
Image.fromarray(mat_contents['X'][n].reshape(28,28),'L') 

6


In [69]:
dataset = np.concatenate((mat_contents['X'], mat_contents['Y']), axis=1) # X, Y = dataset[:,:-1], dataset[:,-1] 

In [92]:
# separate dataset into training and testing

def split_data(dataset, ratio):
    msk = np.random.rand(len(dataset)) < ratio
    train_data = dataset[msk]
    test_data = dataset[~msk]
    return train_data, test_data

train_data, test_data = split_data(dataset, 0.7)

In [216]:
test_data.shape

(2993, 785)

In [ ]:
def percepton_train(train_data, max_iter, digit, mode=3):
        
    w = np.zeros((len(train_data[0]), 1)) # initialize weights and bias (index = 0)        

    if mode == 0: # Perceptron V0
        
        for i in range(max_iter):

            datapoint = train_data[i%(len(train_data)+1)]
            y = 1 if (datapoint[-1] == digit) else -1
            x = datapoint[:-1]

            if y*x.dot(w[1:])[0] + w[0][0] <= 0:

                w[1:] = w[1:] + y*x.reshape(784,1)
                w[0] = w[0] + y

        return w

    if mode == 1: # Perceptron V1
        
        for i in range(max_iter):
        
            datapoint_min = train_data[0]
            y_min = 1 if (datapoint[-1] == digit) else -1
            x_min = datapoint[:-1]
            a_min = y_min*x_min.dot(w[1:])[0] + w[0][0]

            for datapoint in train_data:

                y = 1 if (datapoint[-1] == digit) else -1
                x = datapoint[:-1]
                a = y*x.dot(w[1:])[0] + w[0][0]

                if a < a_min:
                    datapoint_min = datapoint
                    y_min = y
                    x_min = x
                    a_min = a

            if a_min <= 0:
                w[1:] = w[1:] + y_min * x_min.reshape(784,1)
                w[0] = w[0] + y_min

        return w

        
    else:
        
        for i in range(max_iter):
            
            for datapoint in train_data:
                
                y = 1 if (datapoint[-1] == digit) else -1
                x = datapoint[:-1]
                
                if y*x.dot(w[1:])[0] + w[0][0] <= 0:
                    
                    w[1:] = w[1:] + y*x.reshape(784,1)
                    w[0] = w[0] + y
        
        return w



def percepton_test(test_datapoint, w):
    
    return np.sign(test_datapoint[:-1].dot(w[1:])[0] + w[0][0])
    
    

def percepton_accuracy(train_data, test_data, max_iter, digit):
    
    count = 0
    total = len(test_data)
    w = percepton_train(train_data, max_iter, digit)
    
    for datapoint in test_data:
        
        if (percepton_test(datapoint, w) == 1 and datapoint[-1]==digit) \
        or (percepton_test(datapoint, w) == -1 and datapoint[-1]!=digit):
            count += 1
            
    return count/total

In [283]:
for num in range(10):
    print('For digit {}, the accuracy is {}.'.format(num, percepton0_accuracy(train_data, test_data, 100, num)))

For digit 0, the accuracy is 0.9863013698630136.
For digit 1, the accuracy is 0.9856331440026729.
For digit 2, the accuracy is 0.9435349148012028.
For digit 3, the accuracy is 0.937854994988306.
For digit 4, the accuracy is 0.9709321750751754.
For digit 5, the accuracy is 0.9689274974941531.
For digit 6, the accuracy is 0.9752756431673906.
For digit 7, the accuracy is 0.9776144336785834.
For digit 8, the accuracy is 0.9224858002004678.
For digit 9, the accuracy is 0.9492148346140996.


In [ ]:
def percepton1_train(train_data, max_iter, digit):
        
    w = np.zeros((len(train_data[0]), 1)) # initialize weights and bias (index = 0)
    
    for i in range(max_iter):
        
        datapoint_min = train_data[0]
        y_min = 1 if (datapoint[-1] == digit) else -1
        x_min = datapoint[:-1]
        a_min = y_min*x_min.dot(w[1:])[0] + w[0][0]
        
        for datapoint in train_data:
            
            y = 1 if (datapoint[-1] == digit) else -1
            x = datapoint[:-1]
            a = y*x.dot(w[1:])[0] + w[0][0]
            
            if a < a_min:
                datapoint_min = datapoint
                y_min = y
                x_min = x
                a_min = a
                
        if a_min <= 0:
            w[1:] = w[1:] + y_min * x_min.reshape(784,1)
            w[0] = w[0] + y_min
            
    return w

In [289]:
c = np.zeros(len(train_data))

In [293]:
np.sign(-21)

-1

In [305]:
def percepton_train2(train_data, max_iter, digit):
    
    # Perceptron V2  
    
    w = np.zeros((len(train_data[0]), 1)) # initialize weights and bias (index = 0)        
    w_list = [w]
    c = np.zeros(len(train_data))
    k = 1

    for i in range(max_iter):

        datapoint = train_data[i%(len(train_data)+1)]
        y = 1 if (datapoint[-1] == digit) else -1
        x = datapoint[:-1]

        if y*x.dot(w[1:])[0] + w[0][0] <= 0:

            w[1:] = w[1:] + y*x.reshape(784,1)
            w[0] = w[0] + y
            c[k+1] = 1
            k += 1
            w_list.append(w)

        else:

            c[k] += 1

    return w_list, c, k
        
    
def percepton_test2(test_datapoint, w_list, c, k):
    
    sumation = 0
    
    for i in range(k):
        w = w_list[i]
        sumation += c[i] * np.sign(test_datapoint[:-1].dot(w[1:])[0] + w[0][0])
    
    return np.sign(sumation)

In [312]:
np.array(percepton_train2(train_data, 100, 0)).reshape((13,785)).shape

(13, 785)

In [300]:
20%4

0